In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

In [ ]:
dataset_path = '/home/helium/Desktop/VITA/posepred/preprocessed_data/stanford36m/validation_total_stanford3.6m.jsonl'
len_observed = 50
len_future = 25

In [ ]:
from data_loader.random_crop_dataset import RandomCropDataset

dataset = RandomCropDataset(
    dataset_path=dataset_path,
    keypoint_dim=3,
    use_mask=False,
    is_visualizing=False,
    model_pose_format='xyz',
    metric_pose_format='xyz',
    normalize=False,
    metadata_path='',
    seq_rate=1,
    frame_rate=2,
    len_observed=len_observed,
    len_future=len_future,
    is_testing=False,
    is_h36_testing=False
)

In [ ]:
import torch
import numpy as np
from models.st_transformer.CSDI import CSDI_Reconstruction_H36M as MODEL
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from models.st_transformer.st_transformer_loss import STTransformerLoss

device = 'cuda'

def load_snapshot(snapshot_path, MODEL):
    snapshot = torch.load(snapshot_path, map_location='cpu')
    model = MODEL(snapshot['model_args'])
    model.load_state_dict(snapshot['model_state_dict'])
    model = model.to(device)
    return model

def dict_to_device(src, device='cuda'):
    out = dict()
    for key, value in src.items():
        if isinstance(value, torch.Tensor):
            out[key] = value.clone().to(device)
        else:
            out[key] = value
    return out

In [ ]:


model_path = '/home/helium/Desktop/VITA/posepred/outputs/reconstruct/reconstruction_rate1_obs50_pred25_epoch15.pt'
model = load_snapshot(model_path, MODEL)
model.eval()

batch_size=4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
lossFunction = STTransformerLoss(None)

total_loss = 0
sz = 0

for batch in tqdm(dataloader):
    batch = dict_to_device(batch)
    
    model_output = model(batch)
    
    loss = lossFunction(model_output, batch)
    
    total_loss += batch['future_pose'].shape[0] * loss['loss'].cpu().item()
    sz += batch['future_pose'].shape[0]
    
    print("{:.2f}".format(total_loss / sz), end='\r')
    
    